<a href="https://colab.research.google.com/github/fysixs/SPICE/blob/master/SPICEOrbitalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[comment]: <> (Written by Andrés Cárdenas)
[comment]: <> (July 2021)
[comment]: <> (www.fysixs.com)
[comment]: <> (andres.cardenas@gmail.com)

<div class="row" align ="left">

<font face="Gill Sans" color = #667495> **Cool Stuff in Fysixs** <br> *Andrés Cárdenas, 2021*

 <img src="https://drive.google.com/thumbnail?id=1cJN2J8ByFPCfATK70k6CiHeP0bKGN-Ii" alt="Snow" width="35">
 <img src="https://drive.google.com/thumbnail?id=1nmgz_xGqeqgvU8wBfJFERu1Zs82bBZ84" alt="Snow" width="55" align ="right">
 <img src="https://drive.google.com/thumbnail?id=1Jytnbufvmu7v3OWJiiXwVLnLnB8ukJPo" alt="Snow" width="90" align ="right">
  <img src="https://i.imgur.com/7c3Iwcl.png" alt="Snow" height="37" align ="right">
</font>
</div>

<hr size=5 color=#8D84B5 > </hr> 

<div align="left">
<br>

# <font color = #667495 face="Gill Sans"> &nbsp; &nbsp; &nbsp; **SPICE Orbital Analysis**
## <font color = #667495 face="Gill Sans"> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;*Download, plot and analyze the NAIF data directly* </font>

<br>
</div>

<hr size=5 color=#8D84B5 > </hr> 


In [15]:
#@title 🐍 <font size = 3 face="Courier New" color=teal> <b>Environment Setup 💻
# Install spiceypy
print("#### INSTALLING SPICEYPY #####")
!pip install spiceypy
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import spiceypy as spice
import glob
import os
import sys
import pandas as pd
import ipywidgets as widgets
import datetime
from bs4 import BeautifulSoup
from IPython.display import clear_output
from ipywidgets import GridspecLayout


# Obtain the data source list from NAIF

!wget https://naif.jpl.nasa.gov/naif/data_archived.html
!sed -n '/<!--start data-->/{:a;n;/<!--end data-->/b;p;ba}' data_archived.html > /content/table.html

path = '/content/table.html'
   
# empty list
data = []
   
# for getting the header from
# the HTML file
list_header = []
soup = BeautifulSoup(open(path),'html.parser')
header = soup.find_all("table")[0].find("tr")
  
for items in header:
    try:
        list_header.append(items.get_text())
    except:
        continue
  
# for getting the data 
HTML_data = soup.find_all("table")[0].find_all("tr")[1:]
  
for element in HTML_data:
    sub_data = []
    for sub_element in element:
        try:
            if bool(sub_element.a):
              sub_data.append(sub_element.a.get('href'))
            else:
              sub_data.append(sub_element.get_text())
        except:
            continue
    data.append(sub_data)
  
# Storing the data into Pandas
# DataFrame 
spiceDf = pd.DataFrame(data = data, columns = list_header)

clear_output()


<center>
  <img src="https://i.imgur.com/5D0tObE.png" height = 500>
</center>
<div align="right">
  <font align="right" face="Gill Sans"> Data SPICE can give you. <i> Credit: NAIF/JPL</i> <br>
  All NAIF/JPL references at the end </a>. Thank you!
  </font>
</div>



## **Useful sources**



1.   [Fundamental Concepts](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/Tutorials/pdf/individual_docs/04_concepts.pdf)
2.   [SPICE Position](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/spkpos_c.html)

3.   [day of year conversion](https://www.scp.byu.edu/docs/doychart.html)




## **Data**

Import SPICE data, set of ancillary data, from [NAIF](https://https://naif.jpl.nasa.gov/naif/data_archived.html).

In [20]:
#@title **Import data**

# ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
# Operations
# ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

def download_data(dataSubSet, startDate, stopDate, output):
  with output:
    print("#### GETTING DATA FROM NAIF #####")
    # Prepare the directories
    %cd /content/
    %rm -Rf NAIF
    %mkdir NAIF
    %cd NAIF

    dataURL = ( dataSubSet + "&start=" + startDate.value.isoformat() + 
              "&stop=" + stopDate.value.isoformat() + "&action=Subset" )

    !wget  "$dataURL" -O data.zip
    !unzip data.zip
    !source ./*.tcsh

    subDir = dataSubSet.split("/")[-1]
    %cd /content/NAIF/$subDir

    %cp ../*.tm .
    kernel = glob.glob("./*.tm")[0]

    # %cp ../*.TM .
    # kernel = glob.glob("./*.TM")[0]
    
    print("#### CONNECTING KERNELS #####")
    spice.furnsh(kernel)

  output.clear_output()
  return


def import_data():

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  mission_list = [(name[1], name[0]) for name in enumerate(spiceDf['Mission Name'])]

  mission_select = widgets.Dropdown(
      options=mission_list,
      value=0,
      description='Mission:',
  )

  def missionDate(time):
    if time=='start':
      dat = spiceDf['Start Time'][mission_select.value].strip().split('-')
    elif time=='stop':
      dat = spiceDf['Stop Time'][mission_select.value].strip().split('-')
    return datetime.date( int(dat[0]), int(dat[1]), int(dat[2]) )

  startDate = widgets.DatePicker(
      description='Start Date:',
      disabled=False,
      value = missionDate('start')
  )
  
  stopDate = widgets.DatePicker(
      description='Stop Date:',
      disabled=False,
      value = missionDate('stop')
  )

  get_data_button = widgets.Button(description="🛰️ Download data 🛰️",
                                 layout= widgets.Layout( width='auto'),
                                 button_style='warning' )
  
  output = widgets.Output()

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  dataSubSet = spiceDf['Subset Link'][mission_select.value]

  def on_button_clicked(b):
    download_data(dataSubSet, startDate, stopDate, output)
    return
  get_data_button.on_click(on_button_clicked)

  def on_value_change(change):
    startDate.value = missionDate('start')
    stopDate.value  = missionDate('stop')
    return
  mission_select.observe(on_value_change, names='value')

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ  

  grid = GridspecLayout(2, 3,  align_items='flex-start',
                  width='70%')

  grid[0,0] = mission_select
  grid[0,1] = startDate
  grid[0,2] = stopDate
  grid[1,2] = get_data_button

  display(grid)
  display(output)

  return

import_data()


GridspecLayout(children=(Dropdown(description='Mission:', layout=Layout(grid_area='widget001'), options=(('Cas…

Output()

In [ ]:
#@title **Select Plotting Range** { display-mode: "form" }
plotStart = "2008-01-07" #@param {type:"date"}
plotEnd   = "2015-04-22" #@param {type:"date"}
orbiter   = "venus" #@param {type:"string"}
orbiting  = "solar system barycenter" #@param {type:"string"}

def get_data(plotDates, targets, data):
  # Number of time-points to obtain
  steps = data['steps'] #20000

  plotStart = plotDates[0]
  plotEnd   = plotDates[1]

  orbiter  = targets[0]
  orbiting = targets[1]

  # we are going to get positions between these two dates
  utc = [plotStart, plotEnd]

  # Returns seconds since J2000 epoch (January 1, 2000 at 12:00 TerrTime)
  etOne = spice.str2et(utc[0])
  etTwo = spice.str2et(utc[1])
  print("ET Beginning: {}, ET End: {}".format(etOne, etTwo))

  # Creates the array of times requested
  dt = (etTwo-etOne)/steps
  times = [i * dt + etOne for i in range(steps)]

  #Run spkpos to pull location/time data
  #Run spkezr to pull location/vel/time data
  #positions, lightTimes = spice.spkpos(orbiter, times, 'J2000', 'NONE',orbiting)
  states, lightTimes = spice.spkezr(orbiter, times, 'J2000', 'NONE', orbiting)
  # Positions is a 3xsteps vector of XYZ positions
  # Light times is a steps vector of time
  print(states)

  return

def analyze_data():

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Widgets
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  mission_list = [(name[1], name[0]) for name in enumerate(spiceDf['Mission Name'])]

  mission_select = widgets.Dropdown(
      options=mission_list,
      value=0,
      description='Mission:',
  )

  def missionDate(time):
    if time=='start':
      dat = spiceDf['Start Time'][mission_select.value].strip().split('-')
    elif time=='stop':
      dat = spiceDf['Stop Time'][mission_select.value].strip().split('-')
    return datetime.date( int(dat[0]), int(dat[1]), int(dat[2]) )

  startDate = widgets.DatePicker(
      description='Start Date:',
      disabled=False,
      value = missionDate('start')
  )
  
  stopDate = widgets.DatePicker(
      description='Stop Date:',
      disabled=False,
      value = missionDate('stop')
  )

  get_data_button = widgets.Button(description="🛰️ Download data 🛰️",
                                 layout= widgets.Layout( width='auto'),
                                 button_style='warning' )
  
  output = widgets.Output()

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # Interface
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ

  dataSubSet = spiceDf['Subset Link'][mission_select.value]

  def on_button_clicked(b):
    download_data(dataSubSet, startDate, stopDate, output)
    return
  get_data_button.on_click(on_button_clicked)

  def on_value_change(change):
    startDate.value = missionDate('start')
    stopDate.value  = missionDate('stop')
    return
  mission_select.observe(on_value_change, names='value')

  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ
  # UI
  # ΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛΛ  

  grid = GridspecLayout(2, 3,  align_items='flex-start',
                  width='70%')

  grid[0,0] = mission_select
  grid[0,1] = startDate
  grid[0,2] = stopDate
  grid[1,2] = get_data_button

  display(grid)
  display(output)

  return

analyze_data()
 
# And finally plot them
pos = positions.T
fig = go.Figure()
fig.add_trace(go.Scatter3d(x = pos[0], y= pos[1], z = pos[2], mode='lines'))
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0], marker=dict(color='red', size=10)))
fig.update_layout(showlegend=False)
fig.show()
Execution output from May 17, 2021 8:11 PM
	Stream
		ET Beginning: 252936065.18408436, ET End: 482932867.18558073

Code cell <T0pfgCmrVoo1>
#%% [code]
#@title **Select Exra Object** { display-mode: "form" }
orbiter2   = "earth" #@param {type:"string"}

#Run spkpos to pull location/time data
positions2, lightTimes2 = spice.spkpos(orbiter2, times, 'J2000', 'NONE',orbiting)
#states, lightTimes2 = spice.spkezr(orbiter, times, 'J2000', 'NONE',orbiting)
# Positions is a 3xN vector of XYZ positions
# print("Positions: ")
# print(positions[0])
 
# Light times is a N vector of time
# print("Light Times: ")
# print(lightTimes[0])
 
# And finally plot them
pos2 = positions2.T
fig.add_trace(go.Scatter3d(x = pos2[0], y= pos2[1], z = pos2[2], mode='lines'))
fig.update_layout(showlegend=False)
fig.show()

# **Graphs**

The Orbit Component Graph shows the x,y,z position of the orbiter.

In [ ]:
#@title **Orbit Component Graph** { display-mode: "form" }

fig = go.Figure()

x = np.array(times)/60/60/24/30
xCoo = positions.T[0]
yCoo = positions.T[1]
zCoo = positions.T[2]

fig.add_scatter(x=x, y=xCoo, name=r"$x$")
fig.add_scatter(x=x, y=yCoo, name=r"$y$")
fig.add_scatter(x=x, y=zCoo, name=r"$z$")

fig.update_layout(
    title = "Orbit components",
    xaxis_title="time (month)",
    yaxis_title="distance (km)",
)


The radius of the orbit can be calculated by multiplying the speed of light to the normal vector of time.

The local maximums of the r(t) graph is when the distance between moon and earth is furtherst away, also known as apogee. The local minimum points - perigee - are when the earth and moon is closer to each other and it occurs on January 3rd, January 31st and February 28th.

In [ ]:
#@title **r(t)** { display-mode: "form" }
x = np.array(times)/60/60/24/30
y = lightTimes*(299792.458)
fig = go.Figure()
fig.add_scatter(x=x, y=y)  

fig.update_layout(
    title="r(t)",
    xaxis_title="time (month)",
    yaxis_title="length (m)"
)


$$\text{der(j,i)} = \frac{\Delta position}{\Delta time} $$

In [ ]:
def der(j, i): 
    return (positions.T[i][j+1]-positions.T[i][j])/(times[j+1]-times[j])

In [ ]:
#velocity
#@title **velocity** { display-mode: "form" }
vx  = [ der(j,0) for j in range(steps-1) ]
vy  = [ der(j,1) for j in range(steps-1) ]
vz  = [ der(j,2) for j in range(steps-1) ]
timesM = np.array(times)/60/60/24/30
fig = go.Figure()
fig.add_scatter(x=timesM, y=vx, name=r'$v_x$')
fig.add_scatter(x=timesM, y=vy, name=r'$v_y$')
fig.add_scatter(x=timesM, y=vz, name=r'$v_z$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="Component Velocities",
    xaxis_title="time (month)",
    yaxis_title="velocity (km/s)"
)

In [ ]:
#speed
#@title **speed** { display-mode: "form" }
vx = np.array(vx)
vy = np.array(vy)
vz = np.array(vz)
v = np.sqrt(vx**2 + vy**2 + vz**2)
timesM = np.array(times)/60/60/24/30
fig = go.Figure()
fig.add_scatter(x=timesM, y=v, name=r'$v$')
fig.update_layout(
    title="Component Velocities",
    xaxis_title="time (month)",
    yaxis_title="velocity (km/s)"
)

In [ ]:
#@title **acceleration** { display-mode: "form" }
def derX(j, i): 
    return (positions.T[i][j+1]-positions.T[i][j])/(times[j+1]-times[j])
def derV(j, i): 
    if i == 0:
        v = np.copy(vx)
    elif i == 1:
        v = np.copy(vy)
    elif i == 2:
        v = np.copy(vz)
    return (v[j+1]-v[j])/(times[j+1]-times[j])
  ### slope = y2-y1/x2-x1
  ### x = range(7998) y = vx[x]
vx  = [ derX(j,0) for j in range(steps-1) ]
vy  = [ derX(j,1) for j in range(steps-1) ]
vz  = [ derX(j,2) for j in range(steps-1) ]

ax  = [ derV(j,0) for j in range(steps-2) ]
ay  = [ derV(j,1) for j in range(steps-2) ]
az  = [ derV(j,2) for j in range(steps-2) ]
timesM = np.array(times)/60/60/24/30
fig = go.Figure()
fig.add_scatter(x=timesM, y=ax, name=r'$a_x$')
fig.add_scatter(x=timesM, y=ay, name=r'$a_y$')
fig.add_scatter(x=timesM, y=az, name=r'$a_z$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="Component accelerations",
    xaxis_title="time (month)",
    yaxis_title="acceleration (km/s^2)"
)

The angular velocity can be simply calculated by $\frac{speed}{radius}$
Moon's angualr velocity varies throughout its orbit. It travels faster when it is closer to the Earth and slower when it is further away from the Earth. For example, on the graph, date such as February 1st, when the Earth is farthest away from the Moon, has local minimum angular velocity value. Among the peaks, July 3rd has highest angular velocity value (Earth and Sun farthest away.. less gravitational pull from sun allows higher angular velocity??)

In [ ]:
def omega(x):
  speed = np.sqrt(der(x,0)**2 +der(x,1)**2 +der(x,2)**2) 
  radius = lightTimes[x]*3e8
  return speed/radius

In [ ]:
#@title **angular velocity** { display-mode: "form" }
# omega 
angvx = [ omega(k) for k in range(steps-3) ]

fig = go.Figure()
timesM = np.array(times)/60/60/24/30
fig.add_scatter(x=timesM, y=angvx, name=r'$v_x$')
fig.update_layout(
    title = "omega",
    xaxis_title="time (month)",
    yaxis_title="angular velocity (rad/s)",
)

10/26 (when time is 10.8634) something has went through that pulled and made a bump on the graph.. but nothing seemd to happen on the omega graph.

In [ ]:
#@title **angular acceleration** { display-mode: "form" }
#alpha
def deromega(x):
  return (omega(x+1)-omega(x))/(times[x+1]-times[x])
omder = [ deromega(k) for k in range(steps-3) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=omder, name=r'$v_x$')

fig.update_layout(
    title = "alpha",
    xaxis_title="time (month)",
    yaxis_title="acceleration (rad/s^2)"
)
fig.add_annotation(
        x=1.18725,
        y=4.402777e-16,
        xref="x",
        yref="y",
        text="02/05",
        showarrow=True,
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        opacity=0.8
        )

In [ ]:
def planeq(n): #eqaution of plane w/ two vectors
  a     = positions.T[0][n],positions.T[1][n],positions.T[2][n]
  b     = positions.T[0][n+1],positions.T[1][n+1],positions.T[2][n+1]
  cross = np.array([a[1]*b[2]-a[2]*b[1],a[2]*b[0]-a[0]*b[2],a[0]*b[1]-a[1]*b[0]])
  return cross/np.linalg.norm(cross)


In [ ]:
(planeq(0)+planeq(-1))/2

In [ ]:
planeq(10000)

In [ ]:
vecV = (0,0,1)
vecU = (planeq(0)+planeq(-1))/2-planeq(10000)
# angle between  (planeq(0)+planeq(-1))/2-planeq(10000) and (0,0,1)

In [ ]:
magV = np.sqrt(vecV[0]*vecV[0]+ vecV[1]*vecV[1]+vecV[2]*vecV[2])
magU = np.sqrt(vecU[0]*vecU[0]+ vecU[1]*vecU[1]+vecU[2]*vecU[2])
magV , magU

In [ ]:
np.arccos(vecU[2]/(magV * magU)) *180 /3.14

angle between xy plane and the vector 
$$\vec{v} = <0,0,1> \quad \vec{u} = <-0.09147718,  0.3703545 , -0.92449351>$$
 $$cos\theta = \frac{u \cdot v}{\lvert\lvert u \rvert\rvert \text{ } \lvert\lvert v \rvert\rvert } = \frac{-0.92449351}{1*1.0001093814970818} = -0.924392$$
$$\arccos(-0.924392) = 157.577 $$
The smaller angle between the two vector is 22.4$^\circ$

In [ ]:
def ang(k):
    dot  = np.dot(planeq(k), planeq(k+1))
    if(dot>1.0):
        dot = 1.0
    return np.arccos(dot)
#  return np.arccos(np.abs(planeq(k)[0]*planeq(l)[0]+planeq(k)[1]*planeq(l)[1]+planeq(k)[2]*planeq(l)[2])/(np.sqrt(planeq(k)[0]**2+planeq(k)[1]**2+planeq(k)[2]**2)*np.sqrt(planeq(l)[0]**2+planeq(l)[1]**2+planeq(l)[2]**2)))

In [ ]:
#@title **Orbital Vector Angle** { display-mode: "form" }
# Orbital Vector angle
angle = [ ang(k) for k in range(steps-2) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=angle, name=r'$v_x$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="orbital vector angle",
    xaxis_title="time (month)",
    yaxis_title="angle (rad)"
)

In [ ]:
def angvel(k):
  return (ang(k+1)-ang(k))/(times[k+1]-times[k])

In [ ]:
#@title **Angular oscillation freuqnecy of the orbital vector** { display-mode: "form" }
# Angular Oscillation frequency of the orbital vector
# orb vector = perp vector to the orb plane

angvx = [ angvel(k) for k in range(steps-3) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=angvx, name=r'$v_x$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="angular oscillation frequency of the orbital vector",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title"
)

In [ ]:
def angaccel(k):
  return (angvel(k+1)-angvel(k))/(times[k+1]-times[k])

In [ ]:
#@title **Angular Acceleration of the orbit vector** { display-mode: "form" }
# angular acceleration of the orb vec

angvx = [ angaccel(k) for k in range(steps-4) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=angvx, name=r'$v_x$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="angular acceleration of the orbit vector",
    xaxis_title="time (month)",
    yaxis_title="angular acceleration (rad/s^2)"
)

In [ ]:
def F(t):
  #F = mv^2/r
  m = 7.346e22
  v = (lightTimes[t+1]*3e8-lightTimes[t]*3e8)/(times[t+1]-times[t])
  r = lightTimes[t]*(3e8)
  return (m*(v**2))/r

In [ ]:
#@title **centripetal force** { display-mode: "form" }
force = [ F(k) for k in range(steps-1) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=force, name=r'$\dot{r}$)$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(

    title="Force",
    xaxis_title="time (month)",
    yaxis_title="Force (N)"
)

In [ ]:
#@title **3D planeq vector**{ display-mode: "form" }
# create a 3D line plot that follows the tip of planeq vector...
a = [planeq(k) for k in range(steps-1)]
ax = [a[n][0] for n in range(steps-1)]
ay = [a[n][1] for n in range(steps-1)]
az = [a[n][2] for n in range(steps-1)]
fig = go.Figure()
n = steps-2
fig.add_trace(go.Scatter3d(x = ax, y= ay, z = az, mode='lines'))
fig.update_layout(showlegend=False)
fig.update_layout(
    title = "Orbital Plane Vector",
    xaxis_title="time (min)",
    yaxis_title="acceleration (rad/s^2)"
)
fig.show()

In [ ]:
def f(t):
  #F = GMM/R^2
  G = 6.67408e-11  #m3 kg-1 s-2
  M = 5.972e24
  m = 7.346e22
  R = lightTimes[t]*(3e8)
  return (m*M*G)/(R**2)

In [ ]:
#@title **Force** { display-mode: "form" }
force = [ f(k) for k in range(steps-1) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=force, name=r'$\dot{r}$)$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="Force",
    xaxis_title="time (month)",
    yaxis_title="Force (N)"
)

In [ ]:
def dr(t):
  return (lightTimes[t+1]*3e8-lightTimes[t]*3e8)/(times[t+1]-times[t])

In [ ]:
#@title **derivative of the radius of the orbit** { display-mode: "form" }
angvx = [ dr(k) for k in range(steps-1) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=angvx, name=r'$\dot{r}$)$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="derivative of the radius of the orbit",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title"
)

In [ ]:
def ddr(b):
  return (dr(b+1)-dr(b))/(times[b+1]-times[b])

In [ ]:
#@title **ddr** { display-mode: "form" }
ddrvx = [ ddr(k) for k in range(steps-2) ]
fig = go.Figure()
fig.add_scatter(x=timesM, y=ddrvx, name=r'$\ddot{r}$')
timesM = np.array(times)/60/60/24/30
fig.update_layout(
    title="ddr",
    xaxis_title="X Axis Title",
    yaxis_title="Y Axis Title"
)

<hr size=3 color=#8D84B5 align="left"> </hr> 


### <font color = #667495 face="Gill Sans"> *References*</font>
<hr size=2 color=#8D84B5 width=30% align="left"> </hr> 

1. Acton, C.H.; "Ancillary Data Services of NASA's Navigation and Ancillary Information Facility;" Planetary and Space Science, Vol. 44, No. 1, pp. 65-70, 1996.

2. Charles Acton, Nathaniel Bachman, Boris Semenov, Edward Wright; A look toward the future in the handling of space science mission geometry; Planetary and Space Science (2017); 
DOI 10.1016/j.pss.2017.02.013
https://doi.org/10.1016/j.pss.2017.02.013
